In [1]:
from data import get_formatted_datasets
import os

/data/workspace/miniconda3/envs/moa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path ="/data/workspace/projects/moe/datasets/eval_code"
data_name = "humaneval"
data_path_1 = os.path.join(data_path, data_name)
data_name = data_name.lower()
# Load and format datasets
formatted_datasets = get_formatted_datasets(data_path=data_path_1, prompt_only=True)

Datasets: DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'entry_point', 'canonical_solution', 'test'],
        num_rows: 164
    })
})
Example: {'task_id': 'HumanEval/0', 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n', 'entry_point': 'has_close_elements', 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n', 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    a

Map: 100%|██████████| 164/164 [00:00<00:00, 17324.41 examples/s]

Formatted datasets: DatasetDict({
    test: Dataset({
        features: ['task_id', 'prompt', 'entry_point', 'canonical_solution', 'test', 'instruction', 'data_name', 'text'],
        num_rows: 164
    })
})
Formatted example: {'task_id': 'HumanEval/0', 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n', 'entry_point': 'has_close_elements', 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n', 'test': "\n\nMETADATA = {\n    'author': 'jt',\n  

In [33]:
predict_file = '/data/workspace/projects/moe/AdaMoLE/outputs/llama-3-1-8b-instruct-lora-codealpaca20k_3epoch/predictions/humaneval_responses.jsonl'
mocorelora_corerouter_file = '/data/workspace/projects/moe/AdaMoLE/outputs/llama-3-1-8b-instruct-mocorelora-exp8-corerouter-codealpaca20k/predictions/humaneval_responses.jsonl'
mbpp_lora_file = '/data/workspace/projects/moe/AdaMoLE/outputs/llama-3-1-8b-instruct-lora-codealpaca20k_3epoch/predictions/mbpp_responses.jsonl'
mbpp_mocorelora_corerouter_file = '/data/workspace/projects/moe/AdaMoLE/outputs/llama-3-1-8b-instruct-mocorelora-exp8-corerouter-codealpaca20k/predictions/mbpp_responses.jsonl'

In [13]:
from code_utils.env import get_feedback, is_correct
import json

In [40]:
# with open(predict_file, 'r') as f:
with open(mocorelora_corerouter_file, 'r') as f:
# with open(mbpp_lora_file, 'r') as f:
# with open(mbpp_mocorelora_corerouter_file, 'r') as f:
    lines = f.readlines()
predicts = [json.loads(line) for line in lines]

In [35]:
predicts[499]

IndexError: list index out of range

#  eval code from zhenghaoyu

In [4]:
# humaneval
correct_count = 0
for predict in predicts:
    code = predict['response']
    test = predict['test']
    correct = is_correct(code, test)
    if correct:
        correct_count += 1

print(correct_count/len(predicts))

0.6402439024390244


In [ ]:
# mbpp
correct_count = 0
for i,predict in enumerate(predicts):
    code = predict['response']
    test = predict['test_list']
    print(i)
    if i in (125,126):
        continue
    # is_passing, feedback = get_feedback(code, test)
    # if is_passing:
    #     correct_count += 1
    # else:
    #     pass
        # print(feedback)
    
    test = '\n'.join(test)
    correct = is_correct(code, test)
    if correct:
        correct_count += 1

print(correct_count/len(predicts))

In [ ]:
print(is_correct())

# Human eval 代码评测

In [5]:
from human_eval.data import write_jsonl

In [41]:
predicts = [json.loads(line) for line in lines]
samples = []
for x in predicts:
    if type(x['response']) == list:
        for response in x['response']:
            sample = {}
            sample['task_id'] = x['task_id']
            sample['completion'] = response
            samples.append(sample)
    else:
        sample = {}
        sample['task_id'] = x['task_id']
        sample['completion'] = x['response']
        samples.append(sample)
write_jsonl('sample.jsonl', samples)


In [42]:
# ! evaluate_functional_correctness sample.jsonl 
# ! evaluate_functional_correctness sample.jsonl --problem_file=/data/workspace/projects/moe/datasets/mbpp_forhumaneval.jsonl
! evaluate_functional_correctness sample.jsonl --k='"1,5,10"'

Reading samples...
1640it [00:00, 95986.19it/s]
Running test suites...
100%|██████████████████████████████████████| 1640/1640 [00:14<00:00, 112.52it/s]
Writing results to sample.jsonl_results.jsonl...
100%|███████████████████████████████████| 1640/1640 [00:00<00:00, 142562.87it/s]
{'pass@1': np.float64(0.29329268292682925), 'pass@5': np.float64(0.5536924119241191), 'pass@10': np.float64(0.6524390243902439)}


In [46]:
from human_eval.evaluation import evaluate_functional_correctness

In [49]:
result = evaluate_functional_correctness('sample.jsonl',k=[1,5,10])
result
with open('result.json', 'w') as f:
    json.dump(result, f)

Reading samples...


1640it [00:00, 27432.56it/s]


Running test suites...


100%|██████████| 1640/1640 [00:20<00:00, 79.30it/s]


Writing results to sample.jsonl_results.jsonl...


100%|██████████| 1640/1640 [00:00<00:00, 141974.38it/s]


# get mbpp dataset

In [23]:
import datasets
import json

data_source = "Muennighoff/mbpp"

dataset = datasets.load_dataset(data_source)

test = dataset["test"]

In [36]:
filtered = test.filter(lambda x: 11 <= x["task_id"] <= 510)
filtered


Dataset({
    features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
    num_rows: 500
})

In [38]:
filtered.to_json("/data/workspace/projects/moe/datasets/eval_code/mbpp/test.jsonl", lines=True)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 115.03ba/s]


287032

In [34]:
test

Dataset({
    features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
    num_rows: 974
})

## convert mbpp to human eval formart

In [38]:
with open('/data/workspace/projects/moe/datasets/eval_code/mbpp/test.jsonl', 'r') as f:
    lines = f.readlines()
data = [json.loads(line) for line in lines]

In [39]:
data[0]

{'task_id': 11,
 'text': 'Write a python function to remove first and last occurrence of a given character from the string.',
 'code': 'def remove_Occ(s,ch): \r\n    for i in range(len(s)): \r\n        if (s[i] == ch): \r\n            s = s[0 : i] + s[i + 1:] \r\n            break\r\n    for i in range(len(s) - 1,-1,-1):  \r\n        if (s[i] == ch): \r\n            s = s[0 : i] + s[i + 1:] \r\n            break\r\n    return s ',
 'test_list': ['assert remove_Occ("hello","l") == "heo"',
  'assert remove_Occ("abcda","a") == "bcd"',
  'assert remove_Occ("PHP","P") == "H"'],
 'test_setup_code': '',
 'challenge_test_list': ['assert remove_Occ("hellolloll","l") == "helollol"',
  'assert remove_Occ("","l") == ""']}

In [62]:
import re
new_data = []
for x in data:
    new_x = {}
    new_x['task_id'] = x['task_id']
    new_x['prompt'] = ''
    match = re.search(r'assert\s+([a-zA-Z_]\w*)\s*\(', x['test_list'][0])
    func_name = match.group(1)
    new_x['entry_point'] = func_name
    new_x['test'] = 'def check(candidate):\n\n    ' + '\n    '.join(x['test_list']).replace(func_name, 'candidate')
    new_data.append(new_x)
with open('/data/workspace/projects/moe/datasets/eval_code/mbpp/test_forhumaneval.jsonl', 'a') as f:
    for x in new_data:
        f.write(json.dumps(x) + '\n')

In [55]:
import re
s = "assert extract_string(['Python', 'list',)"
match = re.search(r'assert\s+([a-zA-Z_]\w*)\s*\(', s)
if match:
    func_name = match.group(1)
    print(func_name)

extract_string
